
# Analysis Notebook

This notebook can be used to calculate statistical metrics for the data produced. 
*** 

## 0. Setup <a class="anchor" id="0"></a>
This section imports all files and sets up the notebook.

In [522]:
# Import all pacakges
import os
import pandas as pd
import numpy as np
from scipy import stats


In [523]:
# Choose which results to use
results_to_use = ["Adaptive", "Zero"]  # If this is empty, all files in the folder will be used
WARMUP_EPOCHS = 0 # Number of epochs to ignore.
sim_colours = ['blue', 'green', 'red', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']


all_results = os.listdir('results/')

if results_to_use == [""]:
    results_to_use = all_results
# remove DS_Store from the list
if '.DS_Store' in results_to_use:
    results_to_use.remove('.DS_Store')
    
print("Using results: " + str(results_to_use))


Using results: ['Adaptive', 'Zero']


In [524]:
# Create a dictionary. Each key is a set of results. 

all_sim_results = {}

for sim in results_to_use:
    all_sim_results[sim] = {}
    all_sim_results[sim]['configuration'] = pd.read_csv(
        'results/' + sim + '/configuration.txt', header=None, delimiter=":")
    types_of_results = []
    # Get all the filenames in the folder, excluding the extension
    for file in os.listdir('results/' + sim + '/exported_data/'):
        if file.endswith(".npy"):
            types_of_results.append(file[:-4])
    # Create a dictionary for each type of result
    for result in types_of_results:
        all_sim_results[sim][result] = np.load(
            'results/' + sim + '/exported_data/' + result + '.npy')

In [525]:
# Show all the all_sim_results in the notebook
for set_of_results in results_to_use:
    print("Results: " + set_of_results)
    print(all_sim_results[set_of_results]['configuration'])

Results: Adaptive
                                    0                    1
0                             command                  run
1                       num_of_epochs                10000
2                  num_of_simulations                  100
3                           grid_size                    3
4                      queue_capacity                   10
5                     congestion_rate                 0.07
6                       with_hotspots                 True
7                           wage_time                   10
8                      credit_balance                    5
9           shared_auction_parameters                 None
10  adaptive_auction_action_selection   e_greedy_exp_decay
11               bid_calculation_rule               linear
12       adaptive_auction_update_rule        simple_bandit
13             auction_episode_length                   10
14   action_selection_hyperparameters                 None
15    adaptive_auction_discretization 

## 1. Average Total Number of Trips  <a class="anchor" id="1"></a>

This t-test compares the average total number of trips between the two experiments, for all sims.

In [526]:
all_average_number_of_trips = []
for sim in results_to_use:
    all_average_number_of_trips.append(
        all_sim_results[sim]['stat_num_of_trips_per_simulation'])

results = stats.ttest_ind(all_average_number_of_trips[0], all_average_number_of_trips[1])
print(results)
#present the results nicely
print("The average number of trips for the first two simulations are: " + str(np.mean(all_average_number_of_trips[0])) + " and " + str(np.mean(all_average_number_of_trips[1])))
print("the standard deviation of number of trips for the first two simulations are: " + str(np.std(all_average_number_of_trips[0])) + " and " + str(np.std(all_average_number_of_trips[1])))
mean_1 = round(np.mean(all_average_number_of_trips[0]),3)
mean_2 = round(np.mean(all_average_number_of_trips[1]),3)
std_1 = round(np.std(all_average_number_of_trips[0]),3)
std_2 = round(np.std(all_average_number_of_trips[1]),3)
print("in latex:","$(M=",mean_1,", SD=",std_1,")$","$(M=",mean_2,", SD=",std_2,")$")
print("in latex:", "t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=14.941614323335866, pvalue=2.6868412356045133e-34, df=198.0)
The average number of trips for the first two simulations are: 24888.73 and 24560.77
the standard deviation of number of trips for the first two simulations are: 147.5366974687993 and 161.02458538993355
in latex: $(M= 24888.73 , SD= 147.537 )$ $(M= 24560.77 , SD= 161.025 )$
in latex: t( 198.0 )= 14.942 , p= 0.0
The p-value is: 2.6868412356045133e-34
The difference is significant


## 2. Average Congestion of Central Intersection

This t-test is between the 2 experiments, for all sims. It does not process individual epochs

In [527]:
average_congestion_per_intersection = []
for exp in results_to_use:
    average_congestion_per_intersection.append(
        all_sim_results[exp]['stat_average_congestion_per_intersection'])

results = stats.ttest_ind(average_congestion_per_intersection[0][:,1,1], average_congestion_per_intersection[1][:,1,1])
print(results)
print("The average congestion for the first two simulations are: " + str(np.mean(average_congestion_per_intersection[0][:,1,1])) + " and " + str(np.mean(average_congestion_per_intersection[1][:,1,1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")
    
mean_1 = round(np.mean(average_congestion_per_intersection[0][:,1,1]),3)
mean_2 = round(np.mean(average_congestion_per_intersection[1][:,1,1]),3)
std_1 = round(np.std(average_congestion_per_intersection[0][:,1,1]),3)
std_2 = round(np.std(average_congestion_per_intersection[1][:,1,1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")

TtestResult(statistic=-47.938938550178214, pvalue=6.4716605752490096e-111, df=198.0)
The average congestion for the first two simulations are: 0.30803945 and 0.34397810000000006
The p-value is: 6.4716605752490096e-111
The difference is significant
in latex: $(M= 0.308 ,\ SD= 0.006 )$ $(M= 0.344 ,\ SD= 0.005 )$
in latex: $t( 198.0 )= -47.939 , p= 0.0 $


## 3. Average Time Waited <a id="3"></a>

agent, intersection and grid based

In [528]:
######Agent based######
# Average time
average_time_agent = []
for exp in results_to_use:
    average_time_agent.append(
        all_sim_results[exp]['stat_average_time_waited_per_simulation_agent'])
    
results = stats.ttest_ind(average_time_agent[0], average_time_agent[1])
print(results)
print("The average time waited for the first two simulations are: " + str(np.mean(average_time_agent[0])) + " and " + str(np.mean(average_time_agent[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(average_time_agent[0]),3)
mean_2 = round(np.mean(average_time_agent[1]),3)
std_1 = round(np.std(average_time_agent[0]),3)
std_2 = round(np.std(average_time_agent[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")

# Max time
max_time_agent = []
for exp in results_to_use:
    max_time_agent.append(
        all_sim_results[exp]['stat_max_time_waited_per_simulation_agent'])
    
results = stats.ttest_ind(max_time_agent[0], max_time_agent[1])
print(results)
print("The max time waited for the first two simulations are: " + str(np.mean(max_time_agent[0])) + " and " + str(np.mean(max_time_agent[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(max_time_agent[0]),3)
mean_2 = round(np.mean(max_time_agent[1]),3)
std_1 = round(np.std(max_time_agent[0]),3)
std_2 = round(np.std(max_time_agent[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ",\ p=", round(results[1], 3), "$")

TtestResult(statistic=-115.27669120235495, pvalue=1.8423776792123032e-183, df=198.0)
The average time waited for the first two simulations are: 0.23983044444444446 and 0.60510575
The p-value is: 1.8423776792123032e-183
in latex: $(M= 0.24 ,\ SD= 0.006 )$ $(M= 0.605 ,\ SD= 0.031 )$
in latex: $t( 198.0 )= -115.277 , p= 0.0 $
TtestResult(statistic=-106.81933991370087, pvalue=5.179083985862078e-177, df=198.0)
The max time waited for the first two simulations are: 0.6207537777777777 and 1.7008356666666669
The p-value is: 5.179083985862078e-177
in latex: $(M= 0.621 ,\ SD= 0.014 )$ $(M= 1.701 ,\ SD= 0.1 )$
in latex: $t( 198.0 )= -106.819 ,\ p= 0.0 $


In [529]:
######Intersection based - Average Time######

average_time_intersection = []
for exp in results_to_use:
    average_time_intersection.append(
        all_sim_results[exp]['stat_average_time_waited_per_intersection'])
    
for i in range(len(average_time_intersection[0][0])):
    for j in range(len(average_time_intersection[0][0])):
        # print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(average_time_intersection[0][:,i,j], average_time_intersection[1][:,i,j])
        # print(results)
        # print("The average time waited for the first two simulations are: " + str(np.mean(average_time_intersection[0][:,i,j])) + " and " + str(np.mean(average_time_intersection[1][:,i,j])))
        # print("The p-value is: " + str(results[1]))
        mean_1 = round(np.mean(average_time_intersection[0][:,i,j]),3)
        mean_2 = round(np.mean(average_time_intersection[1][:,i,j]),3)
        std_1 = round(np.std(average_time_intersection[0][:,i,j]),3)
        std_2 = round(np.std(average_time_intersection[1][:,i,j]),3)
        print("Intersection [", i, "][", j, "]: ", "Adaptive $(M=",mean_1,",\ SD=",std_1,")$",", Zero $(M=",mean_2,",\ SD=",std_2,"),\ t(", int(results.df), ")=", round(results[0], 2), ",\ p=", round(results[1], 3), "$")
        # print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
        # print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ",\ p=", round(results[1], 3), "$")
        

Intersection [ 0 ][ 0 ]:  Adaptive $(M= 0.027 ,\ SD= 0.005 )$ , Zero $(M= 0.029 ,\ SD= 0.004 ),\ t( 198 )= -2.82 ,\ p= 0.005 $
Intersection [ 0 ][ 1 ]:  Adaptive $(M= 0.233 ,\ SD= 0.02 )$ , Zero $(M= 0.649 ,\ SD= 0.087 ),\ t( 198 )= -46.22 ,\ p= 0.0 $
Intersection [ 0 ][ 2 ]:  Adaptive $(M= 0.027 ,\ SD= 0.005 )$ , Zero $(M= 0.029 ,\ SD= 0.004 ),\ t( 198 )= -2.94 ,\ p= 0.004 $
Intersection [ 1 ][ 0 ]:  Adaptive $(M= 0.342 ,\ SD= 0.038 )$ , Zero $(M= 0.699 ,\ SD= 0.077 ),\ t( 198 )= -41.31 ,\ p= 0.0 $
Intersection [ 1 ][ 1 ]:  Adaptive $(M= 0.906 ,\ SD= 0.05 )$ , Zero $(M= 2.669 ,\ SD= 0.144 ),\ t( 198 )= -115.15 ,\ p= 0.0 $
Intersection [ 1 ][ 2 ]:  Adaptive $(M= 0.284 ,\ SD= 0.027 )$ , Zero $(M= 0.678 ,\ SD= 0.091 ),\ t( 198 )= -41.32 ,\ p= 0.0 $
Intersection [ 2 ][ 0 ]:  Adaptive $(M= 0.026 ,\ SD= 0.006 )$ , Zero $(M= 0.027 ,\ SD= 0.004 ),\ t( 198 )= -1.91 ,\ p= 0.058 $
Intersection [ 2 ][ 1 ]:  Adaptive $(M= 0.288 ,\ SD= 0.035 )$ , Zero $(M= 0.639 ,\ SD= 0.076 ),\ t( 198 )= -41.68 ,\

In [530]:
######Intersection based - Max Time######

max_time_intersection = []
for exp in results_to_use:
    max_time_intersection.append(
        all_sim_results[exp]['stat_max_time_waited_per_intersection'])
    
for i in range(len(max_time_intersection[0][0])):
    for j in range(len(max_time_intersection[0][0])):
        # print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(max_time_intersection[0][:,i,j], max_time_intersection[1][:,i,j])
        mean_1 = round(np.mean(max_time_intersection[0][:,i,j]),3)
        mean_2 = round(np.mean(max_time_intersection[1][:,i,j]),3)
        std_1 = round(np.std(max_time_intersection[0][:,i,j]),3)
        std_2 = round(np.std(max_time_intersection[1][:,i,j]),3)
        print("Intersection [", i, "][", j, "]: ", "Adaptive $(M=",mean_1,",\ SD=",std_1,")$",", Zero $(M=",mean_2,",\ SD=",std_2,"),\ t(", int(results.df), ")=", round(results[0], 2), ",\ p=", round(results[1], 3), "$")
        # print(results)
        # print("The average time waited for the first two simulations are: " + str(np.mean(max_time_intersection[0][:,i,j])) + " and " + str(np.mean(max_time_intersection[1][:,i,j])))
        # print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]:  Adaptive $(M= 0.095 ,\ SD= 0.014 )$ , Zero $(M= 0.105 ,\ SD= 0.012 ),\ t( 198 )= -5.36 ,\ p= 0.0 $
Intersection [ 0 ][ 1 ]:  Adaptive $(M= 0.651 ,\ SD= 0.049 )$ , Zero $(M= 1.917 ,\ SD= 0.272 ),\ t( 198 )= -45.63 ,\ p= 0.0 $
Intersection [ 0 ][ 2 ]:  Adaptive $(M= 0.093 ,\ SD= 0.014 )$ , Zero $(M= 0.104 ,\ SD= 0.013 ),\ t( 198 )= -5.56 ,\ p= 0.0 $
Intersection [ 1 ][ 0 ]:  Adaptive $(M= 0.914 ,\ SD= 0.095 )$ , Zero $(M= 2.061 ,\ SD= 0.247 ),\ t( 198 )= -43.12 ,\ p= 0.0 $
Intersection [ 1 ][ 1 ]:  Adaptive $(M= 2.09 ,\ SD= 0.122 )$ , Zero $(M= 7.016 ,\ SD= 0.462 ),\ t( 198 )= -102.5 ,\ p= 0.0 $
Intersection [ 1 ][ 2 ]:  Adaptive $(M= 0.776 ,\ SD= 0.066 )$ , Zero $(M= 2.019 ,\ SD= 0.296 ),\ t( 198 )= -40.82 ,\ p= 0.0 $
Intersection [ 2 ][ 0 ]:  Adaptive $(M= 0.09 ,\ SD= 0.016 )$ , Zero $(M= 0.099 ,\ SD= 0.012 ),\ t( 198 )= -4.28 ,\ p= 0.0 $
Intersection [ 2 ][ 1 ]:  Adaptive $(M= 0.788 ,\ SD= 0.088 )$ , Zero $(M= 1.886 ,\ SD= 0.239 ),\ t( 198 )= -42.91 ,\ p= 0.0

In [531]:
###### Grid based######

# Average time
print("AVERAGE TIME WAITED")
average_time_grid = []
for exp in results_to_use:
    average_time_grid.append(
        all_sim_results[exp]['stat_average_time_waited_grid'])

means_1 = []
for i in range(len(average_time_grid[0])):
    means_1.append(np.mean(average_time_grid[0][i]))
means_2 = []
for i in range(len(average_time_grid[1])):
    means_2.append(np.mean(average_time_grid[1][i]))

print(np.mean(average_time_grid[0], axis=0))
# print("mean = ", np.mean(np.mean(average_time_grid[0], axis=0)))
# print("std = ", np.std(np.mean(average_time_grid[0], axis=0)))

results = stats.ttest_ind(means_1, means_2)
print(results)
print("The average time waited for the first two simulations are: " +
      str(np.mean(means_1)) + " and " + str(np.mean(means_2)))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(means_1), 3)
mean_2 = round(np.mean(means_2), 3)
std_1 = round(np.std(means_1), 3)
std_2 = round(np.std(means_2), 3)
print("in latex:", "$(M=", mean_1, ",\ SD=", std_1, ")$",
      "$(M=", mean_2, ",\ SD=", std_2, ")$")
print("in latex:", "$t(", results.df, ")=", round(
    results[0], 3), ", p=", round(results[1], 3), "$")










# # Max time
# print("MAX TIME WAITED")
# max_time_grid = []
# for exp in results_to_use:
#     max_time_grid.append(
#         all_sim_results[exp]['stat_max_time_waited_grid'])

# means_1 = []
# for i in range(len(max_time_grid[0])):
#     means_1.append(np.mean(max_time_grid[0][i]))
# means_2 = []
# for i in range(len(max_time_grid[1])):
#     means_2.append(np.mean(max_time_grid[1][i]))


# results = stats.ttest_ind(means_1, means_2)
# print(results)
# print("The max time waited for the first two simulations are: " +
#       str(np.mean(means_1)) + " and " + str(np.mean(means_2)))
# print("The p-value is: " + str(results[1]))
# mean_1 = round(np.mean(means_1), 3)
# mean_2 = round(np.mean(means_2), 3)
# std_1 = round(np.std(means_1), 3)
# std_2 = round(np.std(means_2), 3)
# print("in latex:", "$(M=", mean_1, ",\ SD=", std_1, ")$",
#       "$(M=", mean_2, ",\ SD=", std_2, ")$")
# print("in latex:", "$t(", results.df, ")=", round(
#     results[0], 3), ", p=", round(results[1], 3), "$")

AVERAGE TIME WAITED
[[0.0271565  0.233166   0.0268695 ]
 [0.34171025 0.906314   0.28369175]
 [0.02603975 0.288191   0.02533525]]
TtestResult(statistic=-115.27669120235494, pvalue=1.842377679212408e-183, df=198.0)
The average time waited for the first two simulations are: 0.23983044444444446 and 0.60510575
The p-value is: 1.842377679212408e-183
in latex: $(M= 0.24 ,\ SD= 0.006 )$ $(M= 0.605 ,\ SD= 0.031 )$
in latex: $t( 198.0 )= -115.277 , p= 0.0 $


## 4. Gini coefficient <a id="4"></a>

In [532]:
# Average Time Waited.
#Averaged over epochs and intersections: the gini of each intersection, and average of all the ginis per epoch.
all_ginis_time_waited = []
for sim in results_to_use:
    all_ginis_time_waited.append(
        all_sim_results[sim]['stat_time_waited_gini'])

results = stats.ttest_ind(all_ginis_time_waited[0], all_ginis_time_waited[1])
print(results)
# Present the results nicely
print("The average gini of time waited for the first two simulations are: " + str(np.mean(all_ginis_time_waited[0])) + " and " + str(np.mean(all_ginis_time_waited[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(all_ginis_time_waited[0]),3)
mean_2 = round(np.mean(all_ginis_time_waited[1]),3)
std_1 = round(np.std(all_ginis_time_waited[0]),3)
std_2 = round(np.std(all_ginis_time_waited[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")



TtestResult(statistic=33.412419014122115, pvalue=2.5368267435980213e-83, df=198.0)
The average gini of time waited for the first two simulations are: 0.14131020591026344 and 0.1314799973220571
The p-value is: 2.5368267435980213e-83
in latex: $(M= 0.141 ,\ SD= 0.002 )$ $(M= 0.131 ,\ SD= 0.002 )$
in latex: $t( 198.0 )= 33.412 , p= 0.0 $


In [533]:
# Average Satisfaction.
#Averaged over epochs and intersections: the gini of each intersection, and average of all the ginis per epoch.
all_ginis_satisfaction = []
for sim in results_to_use:
    all_ginis_satisfaction.append(
        all_sim_results[sim]['stat_satisfaction_gini'])

results = stats.ttest_ind(all_ginis_time_waited[0], all_ginis_time_waited[1])
print(results)
#present the results nicely
print("The average gini of satisfaction for the first two simulations are: " + str(np.mean(all_ginis_satisfaction[0])) + " and " + str(np.mean(all_ginis_satisfaction[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(all_ginis_satisfaction[0]),3)
mean_2 = round(np.mean(all_ginis_satisfaction[1]),3)
std_1 = round(np.std(all_ginis_satisfaction[0]),3)
std_2 = round(np.std(all_ginis_satisfaction[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")



TtestResult(statistic=33.412419014122115, pvalue=2.5368267435980213e-83, df=198.0)
The average gini of satisfaction for the first two simulations are: 0.4254760946572833 and 0.444973624021619
The p-value is: 2.5368267435980213e-83
in latex: $(M= 0.425 ,\ SD= 0.002 )$ $(M= 0.445 ,\ SD= 0.002 )$
in latex: $t( 198.0 )= 33.412 , p= 0.0 $


## 5. Trip Satisfaction <a id="5"></a>

In [534]:
all_satisfactions = []
for sim in results_to_use:
    all_satisfactions.append(
        all_sim_results[sim]['stat_satisfaction_mean'])

results = stats.ttest_ind(all_satisfactions[0], all_satisfactions[1])
print(results)
#present the results nicely
print("The average satisfaction for the first two simulations are: " + str(np.mean(all_satisfactions[0])) + " and " + str(np.mean(all_satisfactions[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(all_satisfactions[0]),3)
mean_2 = round(np.mean(all_satisfactions[1]),3)
std_1 = round(np.std(all_satisfactions[0]),3)
std_2 = round(np.std(all_satisfactions[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")


TtestResult(statistic=-26.461828416528476, pvalue=6.18137240128031e-67, df=198.0)
The average satisfaction for the first two simulations are: 0.15633658081673904 and 0.1599203439257372
The p-value is: 6.18137240128031e-67
in latex: $(M= 0.156 ,\ SD= 0.001 )$ $(M= 0.16 ,\ SD= 0.001 )$
in latex: $t( 198.0 )= -26.462 , p= 0.0 $


## 6. Auction Reward <a id="5"></a>

In [535]:
######Intersection based - Average Auction Reward######

average_reward_intersection = []
for exp in results_to_use:
    average_reward_intersection.append(
        all_sim_results[exp]['stat_average_auction_reward_per_intersection'])
    
for i in range(len(average_reward_intersection[0][0])):
    for j in range(len(average_reward_intersection[0][0])):
        print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(average_reward_intersection[0][:,i,j], average_reward_intersection[1][:,i,j])
        print(results)
        print("The average auction reward for the first two simulations are: " + str(np.mean(average_reward_intersection[0][:,i,j])) + " and " + str(np.mean(average_reward_intersection[1][:,i,j])))
        print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]
TtestResult(statistic=10.860884846980843, pvalue=7.410933872025321e-22, df=198.0)
The average auction reward for the first two simulations are: 0.05166941666666667 and 0.046528
The p-value is: 7.410933872025321e-22
Intersection [ 0 ][ 1 ]
TtestResult(statistic=17.44435368049987, pvalue=6.831514697775527e-42, df=198.0)
The average auction reward for the first two simulations are: 0.25268666666666667 and 0.23226274999999993
The p-value is: 6.831514697775527e-42
Intersection [ 0 ][ 2 ]
TtestResult(statistic=13.13635383548528, pvalue=9.31059008586125e-29, df=198.0)
The average auction reward for the first two simulations are: 0.05181191666666665 and 0.046210999999999995
The p-value is: 9.31059008586125e-29
Intersection [ 1 ][ 0 ]
TtestResult(statistic=28.620625885612217, pvalue=2.745988571859279e-72, df=198.0)
The average auction reward for the first two simulations are: 0.2633661666666666 and 0.23000074999999995
The p-value is: 2.745988571859279e-72
Intersection [ 